In [3]:
import pandas as pd  # type: ignore
from pandasql import sqldf # type: ignore
data = pd.read_csv('IndieZ.csv')

## **1\. User experience comparison**

**Main problem**: Identification for user experience's change between 2 versions.

**Hypothesis**: With the given information of bad user experience in the tutorial and my own experience after several test runs, it is apparent that the tutorial section failed to explain the game's rule. I theorized that with the uncertanty of not knowing how to play the game, a good amount of players would struggle at level 1 even though it is supposed to be the easiest level but. To be able to prove this, I compiled a query to first acknowledging the tutorial completion rate.

In [2]:
sqldf('''
SELECT 
    nop.version,
    nop.user as num_of_player,
    notc.user as tut_completed,
    (CAST(notc.user as FLOAT) / CAST(nop.user as FLOAT)) *100 
        as tut_complete_percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as user 
    FROM 
        data
    GROUP BY 
        version
) as nop
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as user
    FROM 
        data
    WHERE 
        quantity = -2 
    GROUP BY 
        version
) as notc
ON nop.version = notc.version 
''')

,version,num_of_player,tut_completed,tut_complete_percentage
0,1.5.2,6671,6341,95.053215
1,1.6.0,6929,6571,94.833309


The tutorial completion rate is fairly similar across both versions. Then using this information, I complied another query to get the number of players that completed the tutorial but lost at level 1:

In [4]:
sqldf('''
SELECT 
    tut_complete.version,
    tut_complete.user as tut_completed_user,
    lv_1_lost.user as lv_1_lost_player,
    ROUND(CAST(lv_1_lost.user as FLOAT) / CAST(tut_complete.user as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as user 
    FROM 
        data
    WHERE 
        quantity = -2 
    GROUP BY 
        version
) as tut_complete
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as user
    FROM 
        data
    WHERE 
        user IN (
            SELECT  
                user
            FROM 
                data
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
        AND level = 1 AND win = 0
    GROUP BY 
        version
) as lv_1_lost
ON tut_complete.version = lv_1_lost.version 
''')

,version,tut_completed_user,lv_1_lost_player,percentage
0,1.5.2,6341,1561,24.62
1,1.6.0,6571,1373,20.89


Using the same logic as the corellation between the uncertanty of the game's rule and level 1's losing rate, I hypothesized that the number of loses would drop significantly in level 2 since most player would understand how the game works by that time:

In [6]:
sqldf('''
SELECT 
    tut_complete.version,
    tut_complete.player as num_of_lv_2_player,
    lost.player as lv_2_lost_player,
    ROUND(CAST(lost.player as FLOAT) / CAST(tut_complete.player as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as player 
    FROM 
        data
    WHERE 
        level = 2
    GROUP BY 
        version
) as tut_complete
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as player
    FROM 
        data
    WHERE 
        user IN (
            SELECT  
                user
            FROM 
                data
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
            AND 
                level = 2 AND win = 0
    GROUP BY 
        version
) as lost
ON tut_complete.version = lost.version
''')

,version,num_of_lv_2_player,lv_2_lost_player,percentage
0,1.5.2,6223,56,0.90
1,1.6.0,6396,48,0.75


As expected, the lost rate dropped dramatically from more than 20% to less than 1% in both versions.